---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
# plot the degree distribution of a graph on a log-log scale
# %matplotlib notebook
# from matplotlib import pyplot as plt
def deg_dist(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    plt.plot(degree_values,histogram,'o')
    plt.xlabel('Degree')
    plt.ylabel('Fraction of Nodes')
    plt.xscale('log')
    plt.yscale('log')
    

In [4]:
def avgClustering(G):
    return nx.average_clustering(G)

In [5]:
def avgShortestPath(G):
    return nx.average_shortest_path_length(G)

In [6]:
# super low clustering is a sign of extremely high probability p
# super high average shortest path is a sign of extremely low probability p
# print(avgClustering(P1_Graphs[2]))
# print(avgShortestPath(P1_Graphs[2]))

0.4018222222222227
9.378702269692925


In [7]:
def graph_identification():
    return ['PA','SW_L','SW_L','PA','SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [8]:
G = nx.read_gpickle('email_prediction.txt')

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [9]:
# # tried logistic regression
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score

# clf = LogisticRegression()
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_train)
# clf.fit(X_train_scaled,y_train)
# pred = clf.predict(X_train_scaled)
# roc_auc_score(y_train,pred)

In [10]:
# # tried svm classification
# from sklearn.svm import SVC
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score

# clf = SVC(kernel='linear',probability=True)
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_train)
# clf.fit(X_train_scaled,y_train)
# pred = clf.predict(X_train_scaled)
# roc_auc_score(y_train,pred)

In [11]:
# # tried neural networks
# from sklearn.neural_network import MLPClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score

# clf = MLPClassifier(hidden_layer_sizes=[10,5],alpha=5,solver='lbfgs')
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# clf.fit(X_train_scaled,y_train)
# pred = clf.predict(X_train_scaled)
# roc_auc_score(y_train,pred)

In [12]:
def salary_predictions():
    G = nx.read_gpickle('email_prediction.txt')
    
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G,normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G,normalized=True))
    df['pagerank'] = pd.Series(nx.pagerank(G))
    df.head()
    
    is_NaN = df.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df[row_has_NaN]
    df = df.dropna()

    # training data is data with ManagementSalary data
    training_data = df
    y_train = df['ManagementSalary'].values
    X_train = df.loc[:,['Department','clustering','degree','degree_centrality','closeness','betweeness','pagerank']].values
    # test data is those with NaN for ManagementSalary
    test_data = rows_with_NaN
    indices = test_data.reset_index()['index'].values
    X_test = test_data.loc[:,['Department','clustering','degree','degree_centrality','closeness','betweeness','pagerank']].values

    # random forest regressor will output a probability between 0 and 1 which is basically the probaiblity of binary classification
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score

    model = RandomForestRegressor()

    # grid search for optimal params
    param_grid = {'n_estimators':[5,10], 'max_depth':[5,10]}
    grid_search = GridSearchCV(model, param_grid, scoring="roc_auc")
    grid_result = grid_search.fit(X_train, y_train)

#     prediction = grid_result.predict(X_train)

#     roc_auc_score(y_train,prediction)

    prediction = grid_result.predict(X_test)
    answer = pd.Series(prediction,index=indices)
    
    return answer

In [13]:
# salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [14]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [15]:
# # Gradient boosting classifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.metrics import roc_auc_score

# clf = GradientBoostingClassifier()
# clf.fit(X_train,y_train)
# pred = clf.predict(X_train)
# roc_auc_score(y_train,pred)

# for actual return value
# pred = clf.predict_proba(X_test)[:,1]
# return pred

In [18]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier

def new_connections_predictions():
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    
    future_connections['pref_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2] for node_pair in future_connections.index]
    future_connections['comm_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) for node_pair in future_connections.index]

    is_NaN = future_connections.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = future_connections[row_has_NaN]
    future_connections = future_connections.dropna()

    train = future_connections
    test = rows_with_NaN
    
    X_test = test.loc[:,['pref_attachment','comm_neighbors']]
    X_train = train.loc[:,['pref_attachment','comm_neighbors']]
    y_train = train['Future Connection']
    indices = test.reset_index()['index'].values

    # random forest regressor will output a probability between 0 and 1 which is basically the probaiblity of binary classification
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score

    model = RandomForestRegressor()

    # grid search for optimal params
    param_grid = {'n_estimators':[5,10], 'max_depth':[5,10]}
    grid_search = GridSearchCV(model, param_grid, scoring="roc_auc")
    grid_result = grid_search.fit(X_train, y_train)

#     prediction = grid_result.predict(X_train)

#     print(roc_auc_score(y_train,prediction))

    prediction = grid_result.predict(X_test)
    answer = pd.Series(prediction,index=indices)
    return answer

In [19]:
new_connections_predictions()

(107, 348)    0.032598
(542, 751)    0.012704
(20, 426)     0.607624
(50, 989)     0.012704
(942, 986)    0.012704
(324, 857)    0.012704
(13, 710)     0.157140
(19, 271)     0.157140
(319, 878)    0.012704
(659, 707)    0.012704
(49, 843)     0.012704
(208, 893)    0.012704
(377, 469)    0.012704
(405, 999)    0.019743
(129, 740)    0.019743
(292, 618)    0.019743
(239, 689)    0.012704
(359, 373)    0.012704
(53, 523)     0.032598
(276, 984)    0.012704
(202, 997)    0.012704
(604, 619)    0.116944
(270, 911)    0.012704
(261, 481)    0.059331
(200, 450)    0.716248
(213, 634)    0.012704
(644, 735)    0.116944
(346, 553)    0.012704
(521, 738)    0.012704
(422, 953)    0.019743
                ...   
(672, 848)    0.012704
(28, 127)     0.974676
(202, 661)    0.012704
(54, 195)     0.999611
(295, 864)    0.012704
(814, 936)    0.012704
(839, 874)    0.012704
(139, 843)    0.012704
(461, 544)    0.012704
(68, 487)     0.012704
(622, 932)    0.012704
(504, 936)    0.019743
(479, 528) 